In [1]:
import sys;
import os;
import torch;
import numpy as np;
import torch.optim as optim;
import torch.nn as nn;
from operator import itemgetter;
from heapq import nsmallest;
import time;
import glob;
import math;
import random;

In [2]:
sys.path.append("../../../src/")
sys.path.append("../../")

In [3]:
import common.utils as U;
import common.opts as opt;
import th.resources.models as models;
import th.resources.calculator as calc;
# import th.resources.train_generator as train_generator;
from th.resources.pruning_tools import filter_pruning, filter_pruner;

In [4]:
import argparse
import common.tlopts as tlopts
from SharedLibs.datestring import genDataTimeStr, getDateStr

In [7]:
#Reproducibility
seed = 42;
random.seed(seed);
np.random.seed(seed);
torch.manual_seed(seed);
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed);
if torch.backends.mps.is_available():
    torch.mps.manual_seed(seed)
# torch.backends.cudnn.deterministic = True;
# torch.backends.cudnn.benchmark = False;

### Data Generator

In [8]:
class TLGenerator():
    #Generates data for Keras
    def __init__(self, samples, labels, options):
        random.seed(42);
        #Initialization
        print(f"length of samples:{len(samples)}")
        self.data = [(samples[i], labels[i]) for i in range (0, len(samples))];
        self.opt = options;
        self.batch_size = options.batchSize;
        self.preprocess_funcs = self.preprocess_setup();
        self.mapdict = dict([('52',1),('56',2),('71',3),('99',4)])

    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.data) / self.batch_size));
        #return len(self.samples);

    def __getitem__(self, batchIndex):
        #Generate one batch of data
        # batchX, batchY = self.generate_batch(batchIndex);
        batchX, batchY = self.generate_batch_select_fixed_class(batchIndex);
        batchX = np.expand_dims(batchX, axis=1);
        batchX = np.expand_dims(batchX, axis=3);
        return batchX, batchY

    def generate_batch(self, batchIndex):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        for i in range(self.batch_size):
            # Training phase of BC learning
            # Select two training examples
            while True:
                sound1, label1 = self.data[random.randint(0, len(self.data) - 1)]
                sound2, label2 = self.data[random.randint(0, len(self.data) - 1)]
                if label1 != label2:
                    break
            sound1 = self.preprocess(sound1)
            sound2 = self.preprocess(sound2)

            # Mix two examples
            r = np.array(random.random())
            sound = U.mix(sound1, sound2, r, self.opt.sr).astype(np.float32)
            # print(f"sound length after U.mix is {len(sound)}")
            eye = np.eye(self.opt.nClasses)
            idx1 = self.mapdict[str(label1)]- 1
            idx2 = self.mapdict[str(label2)] - 1
            label = (eye[idx1] * r + eye[idx2] * (1 - r)).astype(np.float32)
            # label = (eye[label1] * r + eye[label2] * (1 - r)).astype(np.float32)

            #For stronger augmentation
            sound = U.random_gain(6)(sound).astype(np.float32)
            # print(f"sound length after U.random_gain is {len(sound)}")
            sounds.append(sound);
            labels.append(label);

        sounds = np.asarray(sounds);
        labels = np.asarray(labels);
        # print(f"labels in generate_batch is:\n{labels}")

        return sounds, labels;

    def generate_batch_select_fixed_class(self, batchIndex):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        #two variables recording alarm and moaning sounds count
        alarm_selected = 0;
        moaning_selected = 0;
        help_eng_selected = 0;
        for i in range(self.batch_size):
            # Training phase of BC learning
            # Select two training examples
            while True:
                # print("enter while true")
                sound1, label1 = self.data[random.randint(0, len(self.data) - 1)]
                sound2, label2 = self.data[random.randint(0, len(self.data) - 1)]
                lbl1_int = np.int16(label1);
                lbl2_int = np.int16(label2);
                # print(f"label1:{label1} and label2:{label2}")
                # print(f"label1:{type(label1)} and label2:{type(label2)}")
                if (lbl1_int == 52 and lbl2_int == 99) or (lbl1_int == 99 and lbl2_int ==52):
                    # if (alarm_selected < moaning_selected) or (alarm_selected == moaning_selected):
                    if (alarm_selected == moaning_selected) and (alarm_selected == help_eng_selected):
                        alarm_selected += 1;
                        break;
                if (lbl1_int == 56 and lbl2_int == 99) or (lbl1_int == 99 and lbl2_int == 56):
                    if (moaning_selected < alarm_selected) and (moaning_selected == help_eng_selected):
                        moaning_selected += 1;
                        break;
                if (lbl1_int == 71 and lbl2_int == 99) or (lbl1_int == 99 and lbl2_int == 71):
                    if (help_eng_selected < alarm_selected) and (help_eng_selected < moaning_selected):
                        help_eng_selected += 1;
                        break;
            # print(f"escape for loop");
            sound1 = self.preprocess(sound1)
            sound2 = self.preprocess(sound2)

            # Mix two examples
            r = np.array(random.random());
            #######make wanted class mix ration above 0.5##########
            # iLbl1 = np.int16(label1);
            # iLbl2 = np.int16(label2);
            # r = 1.0;
            # p_ratio1 = 0.4
            # p_ratio2 = 0.6
            # while True:
            #     r = np.array(random.random());
            #     if r > p_ratio1 and iLbl1 != 99 :
            #         break;
            #     if r < p_ratio2 and iLbl2 != 99 :
            #         break;
            #######################End#######################
            # print(f"r:{r}, lbl1:{label1}, lbl2:{label2}")  
            sound = U.mix(sound1, sound2, r, self.opt.sr).astype(np.float32)
            eye = np.eye(self.opt.nClasses)
            idx1 = self.mapdict[str(label1)]- 1
            idx2 = self.mapdict[str(label2)] - 1
            label = (eye[idx1] * r + eye[idx2] * (1 - r)).astype(np.float32)
            

            #For stronger augmentation
            sound = U.random_gain(6)(sound).astype(np.float32)
            # print(f"sound length after U.random_gain is {len(sound)}")
            sounds.append(sound);
            labels.append(label);

        sounds = np.asarray(sounds);
        labels = np.asarray(labels);
        # print(f"batchIndex is {batchIndex}, total sounds is {len(sounds)}")
        # print(f"labels in generate_batch is:\n{labels}")
        # print(f"alarm_selected:{alarm_selected}, moaning_selected:{moaning_selected}");
        return sounds, labels;

    def preprocess_setup(self):
        funcs = []
        if self.opt.strongAugment:
            funcs += [U.random_scale(1.25)]

        funcs += [U.padding(self.opt.inputLength // 2),
                  U.random_crop(self.opt.inputLength),
                  U.normalize(32768.0)]
        return funcs

    def preprocess_setup_withoutt_normalize(self):
        funcs = []
        if self.opt.strongAugment:
            funcs += [U.random_scale(1.25)]

        funcs += [U.padding(self.opt.inputLength // 2),
                  U.random_crop(self.opt.inputLength)
                 ]
        return funcs

    def preprocess(self, sound):
        for f in self.preprocess_funcs:
            sound = f(sound)

        return sound;

In [9]:
def getTrainGen(opt=None, split=None):
    dataset = np.load(opt.trainSet, allow_pickle=True);
    train_sounds = []
    train_labels = []
    # train_sounds = [dataset['x'][i][0] for i in range(len(dataset['x']))]
    # train_labels = [dataset['y'][i][0] for i in range(len(dataset['y']))]
    train_sounds = dataset['{}'.format(opt.current_fold)].item()['sounds']
    train_labels = dataset['{}'.format(opt.current_fold)].item()['labels']
    # print(train_sounds)

    trainGen = TLGenerator(train_sounds, train_labels, opt);
    trainGen.preprocess_setup();
    return trainGen

### option object creation

In [10]:
def getOpts():
    parser = argparse.ArgumentParser(description='Transfer Learning for ACDNet');
    parser.add_argument('--netType', default='TLACDNet',  required=False);
    parser.add_argument('--data', default='None',  required=False);
    parser.add_argument('--dataset', required=False, default='uec_iot', choices=['10']);
    parser.add_argument('--BC', default=True, action='store_true', help='BC learning');
    parser.add_argument('--strongAugment', default=True,  action='store_true', help='Add scale and gain augmentation');
    #在ipynb中，不能使用parser.parse，要改用parser.parse_known_args()
    opt, unknown = parser.parse_known_args()
    return opt

### ACDNet Define

In [11]:
class Customed_ACDNetV2(nn.Module):
    def __init__(self, input_length, n_class, sr, ch_conf=None):
        super(Customed_ACDNetV2, self).__init__();
        self.input_length = input_length;
        self.ch_config = ch_conf;

        stride1 = 2;
        stride2 = 2;
        channels = 8;
        k_size = (3, 3);
        n_frames = (sr/1000)*10; #No of frames per 10ms

        sfeb_pool_size = int(n_frames/(stride1*stride2));
        # tfeb_pool_size = (2,2);
        if self.ch_config is None:
            self.ch_config = [channels, channels*8, channels*4, channels*8, channels*8, channels*16, channels*16, channels*32, channels*32, channels*64, channels*64, n_class];
        # avg_pool_kernel_size = (1,4) if self.ch_config[1] < 64 else (2,4);
        fcn_no_of_inputs =  self.ch_config[-1];#n_class #self.ch_config[-1];
        # ch_confing_10 = 512 #8 * 64
        # ch_n_class = n_class
        conv1, bn1 = self.make_layers(1, self.ch_config[0], (1, 9), (1, stride1));
        conv2, bn2 = self.make_layers(self.ch_config[0], self.ch_config[1], (1, 5), (1, stride2));
        conv3, bn3 = self.make_layers(1, self.ch_config[2], k_size, padding=1);
        conv4, bn4 = self.make_layers(self.ch_config[2], self.ch_config[3], k_size, padding=1);
        conv5, bn5 = self.make_layers(self.ch_config[3], self.ch_config[4], k_size, padding=1);
        conv6, bn6 = self.make_layers(self.ch_config[4], self.ch_config[5], k_size, padding=1);
        conv7, bn7 = self.make_layers(self.ch_config[5], self.ch_config[6], k_size, padding=1);
        conv8, bn8 = self.make_layers(self.ch_config[6], self.ch_config[7], k_size, padding=1);
        conv9, bn9 = self.make_layers(self.ch_config[7], self.ch_config[8], k_size, padding=1);
        conv10, bn10 = self.make_layers(self.ch_config[8], self.ch_config[9], k_size, padding=1);
        conv11, bn11 = self.make_layers(self.ch_config[9], self.ch_config[10], k_size, padding=1);
        conv12, bn12 = self.make_layers(self.ch_config[10], self.ch_config[11], (1, 1));
        fcn = nn.Linear(fcn_no_of_inputs, n_class);
        nn.init.kaiming_normal_(fcn.weight, nonlinearity='sigmoid') # kaiming with sigoid is equivalent to lecun_normal in keras

        self.sfeb = nn.Sequential(
            #Start: Filter bank
            conv1, bn1, nn.ReLU(),\
            conv2, bn2, nn.ReLU(),\
            nn.MaxPool2d(kernel_size=(1, sfeb_pool_size))
        );

        tfeb_modules = [];
        self.tfeb_width = int(((self.input_length / sr)*1000)/10); # 10ms frames of audio length in seconds
        tfeb_pool_sizes = self.get_tfeb_pool_sizes(self.ch_config[1], self.tfeb_width);
        p_index = 0;
        for i in [3,4,6,8,10]:
            tfeb_modules.extend([eval('conv{}'.format(i)), eval('bn{}'.format(i)), nn.ReLU()]);

            if i != 3:
                tfeb_modules.extend([eval('conv{}'.format(i+1)), eval('bn{}'.format(i+1)), nn.ReLU()]);

            h, w = tfeb_pool_sizes[p_index];
            if h>1 or w>1:
                tfeb_modules.append(nn.MaxPool2d(kernel_size = (h,w)));
            p_index += 1;

        tfeb_modules.append(nn.Dropout(0.2));
        tfeb_modules.extend([conv12, bn12, nn.ReLU()]);
        h, w = tfeb_pool_sizes[-1];
        if h>1 or w>1:
            tfeb_modules.append(nn.AvgPool2d(kernel_size = (h,w)));
        tfeb_modules.extend([nn.Flatten(), fcn]);

        self.tfeb = nn.Sequential(*tfeb_modules);

        self.output = nn.Sequential(
            nn.Softmax(dim=1)
        );
        

    def forward(self, x):
        x = self.sfeb(x);
        #swapaxes
        x = x.permute((0, 2, 1, 3));
        x = self.tfeb(x);
        y = self.output[0](x);
        return y;

    def make_layers(self, in_channels, out_channels, kernel_size, stride=(1,1), padding=0, bias=False):
        conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias);
        nn.init.kaiming_normal_(conv.weight, nonlinearity='relu'); # kaiming with relu is equivalent to he_normal in keras
        bn = nn.BatchNorm2d(out_channels);
        return conv, bn;

    def get_tfeb_pool_sizes(self, con2_ch, width):
        h = self.get_tfeb_pool_size_component(con2_ch);
        w = self.get_tfeb_pool_size_component(width);
        # print(w);
        pool_size = [];
        for  (h1, w1) in zip(h, w):
            pool_size.append((h1, w1));
        return pool_size;

    def get_tfeb_pool_size_component(self, length):
        # print(length);
        c = [];
        index = 1;
        while index <= 6:
            if length >= 2:
                if index == 6:
                    c.append(length);
                else:
                    c.append(2);
                    length = length // 2;
            else:
               c.append(1);

            index += 1;

        return c;

def GetCustomedACDNetModel(input_len=20150, nclass=4, sr=20000, channel_config=None):
    net = Customed_ACDNetV2(input_len, nclass, sr, ch_conf=channel_config);
    return net;

In [21]:
class PruningTrainer:
    def __init__(self, opt):
        self.opt = opt;
        self.opt.channels_to_prune_per_iteration = 1;
        self.opt.finetune_epoch_per_iteration = 2;
        self.opt.lr=0.001;
        self.opt.schedule = [0.5, 0.8];
        self.opt.prune_type = opt.prun_type; #determine the prunning algo, 1: Magnitude Pruning ;2: tylor-pruning
        # torch.device("cuda:0" if torch.cuda.is_available() else "cpu"); #in office use 
        # self.opt.device = 'cuda:0'#office
        self.opt.device = opt.device;#home
        self.pruner = None;
        self.iterations = 0;
        self.cur_acc = 0.0;
        self.cur_iter = 1;
        self.cur_lr = self.opt.lr;
        self.net = None;
        self.criterion = torch.nn.KLDivLoss(reduction='batchmean');
        self.trainGen = getTrainGen(opt)#train_generator.setup(self.opt, self.opt.split);
        self.testX = None;
        self.testY = None;
        self.load_test_data();

    def PruneAndTrain(self):
        print(f"Start to Prune and Train Using device:{self.opt.device}");
        dir = os.getcwd();
        
        trained_model = "../../../trained_models/step_2_first_stage_pruning/multifold/s2_wprun_4C_fold5_2024072310_prunratio85.0/selected/uec_4C_weight_prun_fold5_haacc_95.62841033935547_valacc94.53551483154297_tracc91.19318181818183_epoch_1078_20240723111906.pt"
    
        state= torch.load(trained_model, map_location=self.opt.device);
        self.net = GetCustomedACDNetModel(channel_config=state["config"]).to(self.opt.device);
        self.net.load_state_dict(state['weight']);#home
        self.net = self.net.to(self.opt.device);
        
        self.pruner = filter_pruning.Magnitude(self.net, self.opt) if self.opt.prune_type == 1 else filter_pruning.Taylor(self.net, self.opt);
        self.validate();
        calc.summary(self.net, (1, 1, self.opt.inputLength), brief=False); # shape of one sample for inferenceing
        # exit();
        #Make sure all the layers are trainable
        for param in self.net.parameters():
            param.requires_grad = True
        self.iterations = self.estimate_pruning_iterations();
        # exit();
        for i in range(1, self.iterations):
            self.cur_iter = i;
            iter_start = time.time();
            print("\nIteration {} of {} starts..".format(i, self.iterations-1), flush=True);
            print("Ranking channels.. ", flush=True);
            prune_targets = self.get_candidates_to_prune(self.opt.channels_to_prune_per_iteration);
            # prune_targets = [(40,3)];
            print("Pruning channels: {}".format(prune_targets), flush=True);
            self.net = filter_pruner.prune_layers(self.net, prune_targets, self.opt.prune_all, self.opt.device);
            calc.summary(self.net, (1, 1, self.opt.inputLength), brief=True); # shape of one sample for inferenceing
            self.validate();
            print("Fine tuning {} epochs to recover from prunning iteration.".format(self.opt.finetune_epoch_per_iteration), flush=True);

            if self.cur_iter in list(map(int, np.array(self.iterations)*self.opt.schedule)):
                self.cur_lr *= 0.1;
            optimizer = optim.SGD(self.net.parameters(), lr=self.cur_lr, momentum=0.9);
            self.train(optimizer, epoches = self.opt.finetune_epoch_per_iteration);
            print("Iteration {}/{} finished in {}".format(self.cur_iter, self.iterations+1, U.to_hms(time.time()-iter_start)), flush=True);
            print("Total channels prunned so far: {}".format(i*self.opt.channels_to_prune_per_iteration), flush=True);
            self.__save_model(self.net);

        calc.summary(self.net, (1, 1, self.opt.inputLength)); # shape of one sample for inferenceing
        self.__save_model(self.net);

    def get_candidates_to_prune(self, num_filters_to_prune):
        self.pruner.reset();
        if self.opt.prune_type == 1:
            self.pruner.compute_filter_magnitude();
        else:
            self.train_epoch(rank_filters = True);
            self.pruner.normalize_ranks_per_layer();

        return self.pruner.get_prunning_plan(num_filters_to_prune);

    def estimate_pruning_iterations(self):
        # get total number of variables from all conv2d featuremaps
        prunable_count = sum(self.get_channel_list(self.opt.prune_all));
        total_count= sum(self.get_channel_list());
        #iterations_reqired = int((prunable_count * self.opt.prune_ratio) / self.opt.channels_to_prune_per_iteration);
        #prune_ratio works with the total number of channels, not only with the prunable channels. i.e. 80% or total will be pruned from total or from only features
        iterations_reqired = int((total_count * self.opt.prune_ratio) / self.opt.channels_to_prune_per_iteration);
        print('Total Channels: {}, Prunable: {}, Non-Prunable: {}'.format(total_count, prunable_count, total_count - prunable_count), flush=True);
        print('No. of Channels to prune per iteration: {}'.format(self.opt.channels_to_prune_per_iteration), flush=True);
        print('Total Channels to prune ({}%): {}'.format(int(self.opt.prune_ratio*100), int(total_count * self.opt.prune_ratio)-1), flush=True);
        print('Total iterations required: {}'.format(iterations_reqired-1), flush=True);
        return iterations_reqired;

    def get_channel_list(self, prune_all=True):
        ch_conf = [];
        if prune_all:
            for name, module in enumerate(self.net.sfeb):
                if issubclass(type(module), torch.nn.Conv2d):
                    ch_conf.append(module.out_channels);

        for name, module in enumerate(self.net.tfeb):
            if issubclass(type(module), torch.nn.Conv2d):
                ch_conf.append(module.out_channels);

        return ch_conf;

    def load_test_data(self):
        if(self.testX is None):
            data = np.load(self.opt.valSet, allow_pickle=True);
            dataX = np.moveaxis(data['x'], 3, 1).astype(np.float32);
            # self.testX = torch.tensor(dataX).cuda(); #in office, use cuda(better) or cpu
            # self.testY = torch.tensor(data['y']).cuda();
            self.testX = torch.tensor(dataX).to(self.opt.device);
            # self.testY = torch.tensor(data['y']).to(self.opt.device);
            self.testY = torch.FloatTensor(data['y']).to(self.opt.device);#at home use apple m2

    #Calculating average prediction (10 crops) and final accuracy
    def compute_accuracy(self, y_pred, y_target):
        with torch.no_grad():
            #Reshape to shape theme like each sample comtains 10 samples, calculate mean and find the indices that has highest average value for each sample
            y_pred = (y_pred.reshape(y_pred.shape[0]//self.opt.nCrops, self.opt.nCrops, y_pred.shape[1])).mean(dim=1).argmax(dim=1);
            y_target = (y_target.reshape(y_target.shape[0]//self.opt.nCrops, self.opt.nCrops, y_target.shape[1])).mean(dim=1).argmax(dim=1);
            # y_target = y_target.cuda();
            y_target =y_target.to(self.opt.device);
            acc = (((y_pred==y_target)*1).float().mean()*100).item();
            # valLossFunc = torch.nn.KLDivLoss();
            loss = self.criterion(y_pred.float().log(), y_target.float()).item();
        return acc, loss;

    def train(self, optimizer = None, epoches=10):
        for i in range(epoches):
            # print("Epoch: ", i);
            self.train_epoch(optimizer);
            self.validate();
        print("Finished fine tuning.", flush=True);

    def train_batch(self, optimizer, batch, label, rank_filters):
        self.net.zero_grad()
        if rank_filters:
            output = self.pruner.forward(batch);
            if self.opt.device == "mps":
                # label = label.cpu() #use apple m2, in office use cuda
                # output = output.cpu() #use apple m2, in office use cuda
                label = label.to(self.opt.device);
                output = output.to(self.opt.device);
            self.criterion(output.log(), label).backward();
        else:
            self.criterion(self.net(batch), label).backward();
            optimizer.step();

    def train_epoch(self, optimizer = None, rank_filters = False):
        if rank_filters is False and optimizer is None:
            print('Please provide optimizer to train_epoch', flush=True);
            exit();
        n_batches = math.ceil(len(self.trainGen.data)/self.opt.batchSize);
        for b_idx in range(n_batches):
            x,y = self.trainGen.__getitem__(b_idx)
            x = torch.tensor(np.moveaxis(x, 3, 1)).to(self.opt.device);
            y = torch.tensor(y).to(self.opt.device);
            self.train_batch(optimizer, x, y, rank_filters);

    def validate(self):
        self.net.eval();
        with torch.no_grad():
            y_pred = None;
            batch_size = (self.opt.batchSize//self.opt.nCrops)*self.opt.nCrops;
            for idx in range(math.ceil(len(self.testX)/batch_size)):
                x = self.testX[idx*batch_size : (idx+1)*batch_size];
                # x = x.type(torch.cuda.FloatTensor);
                x = x.type(torch.FloatTensor).to(self.opt.device);
                scores = self.net(x);
                y_pred = scores.data if y_pred is None else torch.cat((y_pred, scores.data));

            acc, loss = self.compute_accuracy(y_pred, self.testY);
        print('Current Testing Performance - Val: Loss {:.3f}  Acc(top1) {:.3f}%'.format(loss, acc), flush=True);
        self.cur_acc = acc;
        self.net.train();
        return acc, loss;

    def __save_model(self, net):
        net.ch_config = self.get_channel_list();
        dir = os.getcwd();
        fname = self.opt.model_name;
        if os.path.isfile(fname):
            os.remove(fname);
        torch.save({'weight':net.state_dict(), 'config':net.ch_config}, fname);

        
    # def __save_model(self, acc, train_acc, epochIdx, net):
    #     if acc > self.bestAcc:
    #         self.bestAcc = acc;
    #         self.bestAccEpoch = epochIdx +1;
    #         __do_save_model(self, acc, train_acc, self.bestAccEpoch, net);
    #     else:
    #         if acc > 94.0 or train_acc > 85.0: 
    #             __do_save_model(self, acc, train_acc, epochIdx, net);
    #         else:
    #             pass

    # def __do_save_model(self, acc, tr_acc, bestAccIdx, net):
    #     save_model_name = self.opt.model_name.format(self.bestAcc, acc, train_acc, epochIdx, genDataTimeStr());
    #     save_model_fullpath = self.opt.save_dir + save_model_name;
    #     print(f"save model to {save_model_fullpath}")
    #     torch.save({'weight':net.state_dict(), 'config':net.ch_config}, save_model_fullpath);
    #     logObj.write(f"save model:{model_name}, bestAcc:{self.bestAcc}@{self.}, currentAcc:{acc}@{epochIdx}");
    #     logObj.write("\n");
    #     logObj.flush();

In [22]:
"""save and record the training hyperparameters and results
pruning algo: tylor-pruning
pruning ration : 0.85
final accuracy : 
epoch: 
self.opt.LR = 0.01;
opt.momentum = 0.009;
self.opt.schedule = [0.15, 0.30, 0.45, 0.60, 0.75];
self.opt.warmup = 0;
self.opt.prune_algo = 'tylor-pruning';
self.opt.prune_interval = 1;
self.opt.nEpochs = 1000;
===============================================
prune_type = Magnitude Pruning
pruning ration : 0.85
final accuracy : 
epoch: 
self.opt.LR = 0.01;
opt.momentum = 0.009;
self.opt.schedule = [0.15, 0.30, 0.45, 0.60, 0.75];
self.opt.warmup = 0;
self.opt.prune_algo = 'tylor-pruning';
self.opt.prune_interval = 1;
self.opt.nEpochs = 1000;
=============================================
"""

"save and record the training hyperparameters and results\npruning algo: tylor-pruning\npruning ration : 0.85\nfinal accuracy : \nepoch: \nself.opt.LR = 0.01;\nopt.momentum = 0.009;\nself.opt.schedule = [0.15, 0.30, 0.45, 0.60, 0.75];\nself.opt.warmup = 0;\nself.opt.prune_algo = 'tylor-pruning';\nself.opt.prune_interval = 1;\nself.opt.nEpochs = 1000;\n===============================================\nprune_type = Magnitude Pruning\npruning ration : 0.85\nfinal accuracy : \nepoch: \nself.opt.LR = 0.01;\nopt.momentum = 0.009;\nself.opt.schedule = [0.15, 0.30, 0.45, 0.60, 0.75];\nself.opt.warmup = 0;\nself.opt.prune_algo = 'tylor-pruning';\nself.opt.prune_interval = 1;\nself.opt.nEpochs = 1000;\n=============================================\n"

In [23]:
def main():
    opt = getOpts()
    #Learning settings
    opt.batchSize = 64;
    #set train and validation sets
    # opt.trainSet = "../../../../uec_iot_models_datasets/version11/single_fold_train_20240603063535.npz" #office
    # opt.valSet = "../../../../uec_iot_models_datasets/version11/final_single_val_20240603063755.npz" #office
    opt.trainSet = "../../../../uec_iot_dev_training_datasets/generated_datasets/multifold/train/version15_multifold_home_fold5/fold5_train_20240721013721.npz";#home
    opt.valSet = "../../../../uec_iot_dev_training_datasets/generated_datasets/multifold/val/version15_multifold_home_fold5/final_fold5_val_version15_multifold_home_20240721024402.npz";#home
    #Basic Net Settings
    opt.prune_ratio = 0.85
    opt.prune_all = True;
    opt.prun_type = 2; #determine the prunning algo, 1: Magnitude Pruning ;2: tylor-pruning
    opt.nClasses = 4
    opt.nFolds = 1;
    opt.split = [i for i in range(1, opt.nFolds + 1)];
    opt.inputLength = 20150;
    #Test data
    opt.nCrops = 2;
    opt.sr = 20000;
    opt.trainer = None
    if torch.backends.mps.is_available():
        opt.device="mps"; #for apple m2 gpu
    elif torch.cuda.is_available():
        opt.device="cuda:0"; #for nVidia gpu
    else:
        opt.device="cpu"
    # opt.device = 'mps';#home
    # tlopts.display_info(opt)
    opt.current_fold='fold5';
    save_dir = "../../../trained_models/step_4_second_stage_pruning/multifold/s3_prun_{}_4C_{}_prunratio{}/".format(opt.current_fold,getDateStr(),opt.prune_ratio*100)
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    # "uec_4C_weight_prun_{}_".format(opt.current_fold)+"haacc_{}_valacc{}_tracc{}_epoch_{}_{}.pt"
    model_name = "uec_4C_IterPrun_{}_ratio{}_{}.pt".format(opt.current_fold, (opt.prune_ratio*100), genDataTimeStr());
    opt.model_name = save_dir + model_name;

    print(f"save model full path:{opt.model_name}")
    # valid_path = False;
    print("Initializing PruneAndTrain Object.....")
    trainer = PruningTrainer(opt=opt)#TLTrainer(opt)
    print("Start to pruning.....")
    trainer.PruneAndTrain();

In [24]:
main()

save model full path:../../../trained_models/step_4_second_stage_pruning/multifold/s3_prun_fold5_4C_2024072902_prunratio85.0/uec_4C_IterPrun_fold5_ratio85.0_20240729023606.pt
Initializing PruneAndTrain Object.....
length of samples:704
Start to pruning.....
Start to Prune and Train Using device:mps
Current Testing Performance - Val: Loss nan  Acc(top1) 94.536%
+----------------------------------------------------------------------------+
+                           Pytorch Model Summary                            +
------------------------------------------------------------------------------
   Layer (type)       Input Shape      Output Shape    Param #      FLOPS #
       Conv2d-1     (1, 1, 20150)     (8, 1, 10071)         72      725,112
  BatchNorm2d-2     (8, 1, 10071)     (8, 1, 10071)         16            0
         ReLu-3     (8, 1, 10071)     (8, 1, 10071)          0       80,568
       Conv2d-4     (8, 1, 10071)     (64, 1, 5034)      2,560   12,887,040
  BatchNorm2d-5     